In [104]:
# imports
import pandas as pd
import time
import os
import requests
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

# get the api key
API_KEY = os.getenv("BLAND_API_KEY")
BASSPRO_PATHWAY_ID = os.getenv("BASSPRO_PATHWAY_ID")

# API constants

In [105]:
BASE_URL = "https://api.bland.ai/v1"
# Headers for authentication
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Obtain all historical calls

In [136]:
# method to get all historical calls we have done
def get_past_calls(start_date: str, end_date: str) -> pd.DataFrame:
    """
    Retrieves past calls within the specified date range.
    """
    url = f"{BASE_URL}/calls?ascending=true"
    params = {
        "start_date": start_date,
        "end_date": end_date
    }

    response = requests.get(url, headers=HEADERS, params=params)

    if response.status_code == 200:
        calls = response.json()["calls"]
        calls_df = pd.DataFrame(calls)
        return calls_df
    else:
        print(f"Failed to retrieve calls: {response.status_code} - {response.text}")
        return []
    
historical_calls_df = get_past_calls(start_date="2024-01-01", end_date="2025-02-28")
historical_calls_df.head()

,c_id,created_at,call_length,to,from,inbound,max_duration,metadata,endpoint_url,variables,...,call_ended_by,analysis,analysis_schema,campaign_id,transferred_to,pathway_tags,status,recording_expiration,pathway_id,call_id
0,b4d25b71-5554-406d-a536-593a7d2c9aab,2025-01-27T00:54:39.075Z,0.950000,+18573662214,+15642131620,False,12,[],deprecated,"{'data': '{""metadata"":[]}', 'now': 'Sunday, Ja...",...,ASSISTANT,None,None,None,None,[],completed,None,6f71c8e3-0798-4b42-8095-e81a3a790a43,b4d25b71-5554-406d-a536-593a7d2c9aab
1,f19c6f0f-c488-4182-99f7-7376ad8be947,2025-01-27T02:43:43.593Z,1.516667,+19076441400,+18646591976,False,12,[],deprecated,"{'data': '{""metadata"":[]}', 'to': '+1907644140...",...,ASSISTANT,None,None,None,None,[],completed,None,6f71c8e3-0798-4b42-8095-e81a3a790a43,f19c6f0f-c488-4182-99f7-7376ad8be947
2,b43c8fa5-0335-4191-8dab-ac00f396f99a,2025-01-27T02:43:57.542Z,1.000000,+19073748800,+15717891373,False,12,[],deprecated,"{'data': '{""metadata"":[]}', 'now': 'Sunday, Ja...",...,ASSISTANT,None,None,None,None,[],completed,None,6f71c8e3-0798-4b42-8095-e81a3a790a43,b43c8fa5-0335-4191-8dab-ac00f396f99a
3,18f5746e-e6de-4155-9ef8-d4c92d3aeac5,2025-01-27T02:44:07.934Z,1.066667,+19075009720,+15204413898,False,12,[],deprecated,"{'data': '{""metadata"":[]}', 'now': 'Sunday, Ja...",...,ASSISTANT,None,None,None,None,[],completed,None,6f71c8e3-0798-4b42-8095-e81a3a790a43,18f5746e-e6de-4155-9ef8-d4c92d3aeac5
4,abaa48dd-96a5-475a-b19b-840a53bed7a8,2025-01-27T02:44:30.666Z,1.833333,+19074203000,+14155238741,False,12,[],deprecated,"{'data': '{""metadata"":[]}', 'now': 'Sunday, Ja...",...,ASSISTANT,None,None,None,None,[],completed,None,6f71c8e3-0798-4b42-8095-e81a3a790a43,abaa48dd-96a5-475a-b19b-840a53bed7a8


# Obtain all historical summaries

In [128]:
def get_past_summaries() -> pd.DataFrame:
    """
    method to get all historical summaries we have generated from calls
    """
    path = "data/bland/summaries"

    # get all the files in the path
    files = os.listdir(path)

    # get the latest file
    # note: files are stored in the format of "summaries_YYYY-MM-DD_HH-MM-SS.xlsx"
    if len(files) > 0:
        latest_file = max(files)
    else:
        print("[ WARN ] No files found in the summaries directory.")
        latest_file = None

    # read the latest file
    if latest_file is not None:
        df =  pd.read_excel(f"{path}/{latest_file}")
        # if there is a column called "to_x", rename it to "to"
        if "to_x" in df.columns:
            df.rename(columns={"to_x": "to"}, inplace=True)
        if "created_at_x" in df.columns:
            df.rename(columns={"created_at_x": "created_at"}, inplace=True)
        if "call_length_x" in df.columns:
            df.rename(columns={"call_length_x": "call_length"}, inplace=True)
        # drop any columns ending in _x or _y
        df = df.loc[:, ~df.columns.str.endswith('_x')]
        df = df.loc[:, ~df.columns.str.endswith('_y')]
        # drop duplicate columns
        df = df.loc[:, ~df.columns.duplicated()]
        return df
    else:
        print("[ WARN ] No historical summaries found.")
        return None

historical_summaries_df = get_past_summaries()
if historical_summaries_df is None:
    pass
historical_summaries_df.head()

,call_id,Does the store have the rifle in stock?,Has the rifle been popular?,Is the rifle sold out?,Is the rifle typically carried at the store?,Is the store restocking the rifle soon?,successful_call,created_at,call_length,to
0,b4d25b71-5554-406d-a536-593a7d2c9aab,NaN,NaN,NaN,NaN,NaN,False,2025-01-27T00:54:39.075Z,0.950000,1.857366e+10
1,f19c6f0f-c488-4182-99f7-7376ad8be947,no,unknown,unknown,no,unknown,True,2025-01-27T02:43:43.593Z,1.516667,1.907644e+10
2,b43c8fa5-0335-4191-8dab-ac00f396f99a,NaN,NaN,NaN,NaN,NaN,False,2025-01-27T02:43:57.542Z,1.000000,1.907375e+10
3,18f5746e-e6de-4155-9ef8-d4c92d3aeac5,no,unknown,unknown,unknown,unknown,True,2025-01-27T02:44:07.934Z,1.066667,1.907501e+10
4,abaa48dd-96a5-475a-b19b-840a53bed7a8,NaN,NaN,NaN,NaN,NaN,False,2025-01-27T02:44:30.666Z,1.833333,1.907420e+10


# Summarize unsummarized calls

In [129]:
def get_unsummarized_call_ids(historical_calls_df: pd.DataFrame, historical_summaries_df: pd.DataFrame) -> list:
    """
    method to get all call_ids from historical_calls_df that are not in historical_summaries_df
    """
    if historical_calls_df is None:
        print("[ WARN ] No historical calls found.")
        return []
    elif historical_summaries_df is None:
        return historical_calls_df["call_id"].unique()
    else:
        # get all call_ids from historical_calls_df that are not in historical_summaries_df
        all_call_ids = historical_calls_df["call_id"].unique()
        summarized_call_ids = historical_summaries_df["call_id"].unique()
        unsummarized_call_ids = [id for id in all_call_ids if id not in summarized_call_ids]
        return unsummarized_call_ids

unsummarized_call_ids = get_unsummarized_call_ids(historical_calls_df, historical_summaries_df)
if unsummarized_call_ids is not None:
    print(f"[ INFO ] Found {len(unsummarized_call_ids)} unsummarized calls.")
else:
    print("[ WARN ] No unsummarized calls found.")

[ INFO ] Found 1 unsummarized calls.


In [130]:
def was_call_successful(summary: dict) -> bool:
    """
    method to determine if the call was successful
    """
    if len(summary["answers"]) == 0:
        return False
    else:
        answers = summary["answers"]
        # if there is at least one answer that is not "unknown", then the call was successful
        for answer in answers:
            if answer != "unknown" and answer != None:
                return True
        return False

def summarize_call(call_id, goal, questions):
    url = f"https://api.bland.ai/v1/calls/{call_id}/analyze"
    payload = {
        "goal": goal,
        "questions": questions
    }
    
    response = requests.post(url, json=payload, headers=HEADERS)

    if response.status_code == 200:
        data = response.json()
        
        if data["status"] == "success":
            summary = {
                "call_id": call_id,
                "questions": [q[0] for q in questions],  # Extract just the question text
                "answers": data["answers"]
            }
            summary["successful_call"] = was_call_successful(summary)
            return summary
        else:
            print(f"Error analyzing call {call_id}: {data['message']}")
            return None
    else:
        print(f"API request failed for call {call_id} with status code {response.status_code}")
        return None

In [132]:
GOAL = """You are summarizing a call between a customer and a gun store employee. 
The customer is inquiring about the availability of a rifle called the 1854 rifle. 
You are to determine if the store had the rifle in stock or not at the time of the call. 
If so, does the store employee indicate that the rifle has been popular? 
If they do not have it in stock, is it because they are sold out? 
Because they have never carreid it? 
If they don't have it in stock, are they restocking soon?

EXAMPLE:

"Yes, we have one in a 64 caliber in stock right now."
"Oh, you have one. Has it been popular?"
"It's been on and off"

YOUR ANSWERS WOULD HAVE BEEN:

1. Did the store indicate that the rifle was in stock? - yes
2. Did the store indicate that the rifle has been popular? - yes
3. Did the store indicate that the rifle is sold out? - no
4. Did the store indicate that the rifle is typically carried at the store? - unknown
5. Did the store indicate that they are restocking the rifle soon? - unknown
"""
QUESTIONS = [
        ["Did the store indicate that the rifle was in stock?","yes, no, unknown"],
        ["Did the store indicate that the rifle has been popular?","yes, no, unknown"],
        ["Did the store indicate that the rifle is sold out?","yes, no, unknown"],
        ["Did the store indicate that the rifle is typically carried at the store?","yes, no, unknown"],
        ["Did the store indicate that they are restocking the rifle soon?","yes, no, unknown"]
    ]

call_summaries = []
for call_id in unsummarized_call_ids:
    try:
        call_summary = summarize_call(call_id, GOAL, QUESTIONS)
        if call_summary:
            # Reshape the data into a flat dictionary
            flat_summary = {
                'call_id': call_summary['call_id']
            }
            # Add each question-answer pair as a column-value
            for q, a in zip(call_summary['questions'], call_summary['answers']):
                flat_summary[q] = a
            flat_summary["successful_call"] = call_summary["successful_call"]
            call_summaries.append(flat_summary)
        else:
            raise Exception()
    except Exception as e:
        print(f"[ WARN ] Error summarizing call {call_id}: {e}")
        time.sleep(30)
        continue

# convert to a dataframe
call_summaries_df = pd.DataFrame(call_summaries)
call_summaries_df.head()

,call_id,Is the rifle in stock?,Has the rifle been popular?,Is the rifle sold out?,Is the rifle typically carried at the store?,Is the store restocking the rifle soon?,successful_call
0,d6e98ea1-6630-47a9-a822-d009fc3ece23,no,null,null,no,null,True


In [133]:
def write_new_summaries(summaries_df: pd.DataFrame, historical_calls_df: pd.DataFrame=None, historical_summaries_df: pd.DataFrame=None) -> None:
    """
    method to write a new summaries file
    """
    if (summaries_df is None or summaries_df.empty) and (historical_summaries_df is None or historical_summaries_df.empty):
        print("[ WARN ] No summaries to write.")
        return
    
    path = "data/bland/summaries"
    date_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # Drop duplicates based on call_id before concatenation
    if historical_summaries_df is not None:
        historical_summaries_df = historical_summaries_df.drop_duplicates(subset=['call_id'])
        if summaries_df is not None:
            summaries_df = summaries_df.drop_duplicates(subset=['call_id'])
            summaries_df = pd.concat([historical_summaries_df, summaries_df], ignore_index=True)
    
    if historical_calls_df is not None:
        summaries_df = summaries_df.merge(
            historical_calls_df[["call_id", "created_at", "call_length", "to"]],
            on="call_id",
            how="left"
        )
    
    summaries_df.to_excel(f"{path}/summaries_{date_str}.xlsx", index=False)

write_new_summaries(call_summaries_df, historical_calls_df, historical_summaries_df)

# Re-send unsuccessful calls

In [134]:
from src.bland import Bland

bland = Bland(api_key=API_KEY)

DEFAULT_KWARGS = {
    "phone_number": None,
    "pathway_id": BASSPRO_PATHWAY_ID,
    "voice": "nat",
    "wait_for_greeting": True
}

In [88]:
# Get all phone numbers from the summaries df that were unsuccessful
historical_summaries_df = get_past_summaries()
if historical_summaries_df is None or historical_summaries_df.empty:
    print("[ WARN ] No historical summaries found.")
    unsuccessful_calls_list = []
else:
    if "to" in historical_summaries_df.columns:
        unsuccessful_calls_list = historical_summaries_df[historical_summaries_df["successful_call"] == False]["to"].unique()
    elif "to_x" in historical_summaries_df.columns:
        unsuccessful_calls_list = historical_summaries_df[historical_summaries_df["successful_call"] == False]["to_x"].unique()
    print(f"[ INFO ] Found {len(unsuccessful_calls_list)} unsuccessful calls.")

# Send these calls again
if len(unsuccessful_calls_list) > 0:
    for phone_number in unsuccessful_calls_list:
        kwargs = DEFAULT_KWARGS.copy()
        kwargs["phone_number"] = phone_number
        _ = bland.call(**kwargs)
else:
    print("[ WARN ] No unsuccessful calls found.")

[ INFO ] Found 82 unsuccessful calls.
{'phone_number': 15094870700.0, 'pathway_id': '6f71c8e3-0798-4b42-8095-e81a3a790a43', 'voice': 'nat', 'wait_for_greeting': True, 'model': 'enhanced'}
{'authorization': 'org_918c6ada38a2d32ef3df9e69f1588554e928230150c00d05ca0554ffeb93b66e67358c59571478c7eda569', 'Content-Type': 'application/json'}
{'phone_number': 13072094500.0, 'pathway_id': '6f71c8e3-0798-4b42-8095-e81a3a790a43', 'voice': 'nat', 'wait_for_greeting': True, 'model': 'enhanced'}
{'authorization': 'org_918c6ada38a2d32ef3df9e69f1588554e928230150c00d05ca0554ffeb93b66e67358c59571478c7eda569', 'Content-Type': 'application/json'}
{'phone_number': 14256102100.0, 'pathway_id': '6f71c8e3-0798-4b42-8095-e81a3a790a43', 'voice': 'nat', 'wait_for_greeting': True, 'model': 'enhanced'}
{'authorization': 'org_918c6ada38a2d32ef3df9e69f1588554e928230150c00d05ca0554ffeb93b66e67358c59571478c7eda569', 'Content-Type': 'application/json'}
{'phone_number': 13602078400.0, 'pathway_id': '6f71c8e3-0798-4b42-80

Exception: {"status":"error","message":"Insufficient balance."}

# Send new calls

In [152]:
# Get all phone numbers from the target csv
target_df = pd.read_csv("data/bass_pro_vf.csv")
all_new_phone_numbers = target_df["phone_number"].tolist()
# Get all phone numbers we have already called
historical_calls_df = get_past_calls(start_date="2024-01-01", end_date="2025-02-28")
old_phone_numbers = historical_calls_df["to"].unique().tolist()
# Separate the ones we have never called
new_phone_numbers_to_call = [phone_number for phone_number in all_new_phone_numbers if phone_number not in old_phone_numbers]

print(f"[ INFO ] Found {len(new_phone_numbers_to_call)} new phone numbers to call.")

[ INFO ] Found 121 new phone numbers to call.


In [153]:
# send the new calls
for phone_number in new_phone_numbers_to_call:
    kwargs = DEFAULT_KWARGS.copy()
    kwargs["phone_number"] = phone_number
    try:
        _ = bland.call(**kwargs)
    except Exception as e:
        print(f"[ WARN ] Error calling {phone_number}: {e}")
        time.sleep(60)
        continue


{'phone_number': '+13168543130', 'pathway_id': '4e2554fa-d1ca-446e-8b3d-8d62ac64ee45', 'voice': 'nat', 'wait_for_greeting': True, 'model': 'enhanced'}
{'authorization': 'org_918c6ada38a2d32ef3df9e69f1588554e928230150c00d05ca0554ffeb93b66e67358c59571478c7eda569', 'Content-Type': 'application/json'}
[ WARN ] Error calling +13168543130: {"status":"error","message":"Rate limit exceeded"}


KeyboardInterrupt: 